# Task 1 DF

In [0]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from sparkmeasure import StageMetrics
from pyspark.sql import functions as F
from pyspark import SparkConf, SparkContext

from operator import add

spark = SparkSession.builder.appName("task1").getOrCreate()

stagemetrics = StageMetrics(spark)

dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"


In [0]:
# https://stackoverflow.com/questions/55453101/pyspark-error-analysisexception-cannot-resolve-column-name
def clean_column_names(df):
  tempList = [] #Edit01
  for col in df.columns:
      new_name = col.strip()
      new_name = "".join(new_name.split())
      new_name = new_name.replace('.','') 
      tempList.append(new_name) 

  return df.toDF(*tempList) 


In [0]:
rename_model = udf(lambda x: x[0:3] if x[0].isdigit() else x[1:4])

def task_1_df(spark_session, flights_path, aircrafts_path, show_results = True):
    flights_df = spark.read.format("csv") \
                        .option("header", "true") \
                        .option("inferSchema", "true") \
                        .load(flights_path)
    aircrafts_df = spark.read.format("csv") \
                        .option("header", "true") \
                        .option("inferSchema", "true") \
                        .load(aircrafts_path)
    
    flights = clean_column_names(flights_df)
    aircraft = clean_column_names(aircrafts_df)
    
    df = aircraft.join(flights, aircraft.tailnum == flights.tail_number, how="inner")
    
    cessna_df = df.filter(F.col('manufacturer') == "CESSNA")
    cessna_updated_df = cessna_df.withColumn("short_model", rename_model(F.col('model')))
    
    ranked_df = cessna_updated_df.groupBy('short_model').count().orderBy('count',ascending=False)
    
    if show_results:
      for row in ranked_df.take(3):
        model = list(row)[0]
        num_outbound_flights = list(row)[1]
        print("Cessna " + model + "\t" + str(num_outbound_flights))

    ranked_df.collect()

In [0]:
task_1_df(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_small.csv", f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv")

Cessna 172	57
Cessna 210	48
Cessna 421	47

In [0]:
task_1_df(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_medium.csv", f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv")

Cessna 210	500
Cessna 172	444
Cessna 421	430

In [0]:
task_1_df(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_large.csv", f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv")

Cessna 210	4716
Cessna 172	4215
Cessna 421	4020